# Simulated Annealing

In [1]:
##This is an initialization cell. Run this first
import pandas as pd
import numpy as np
from itertools import product
import time
import math
import matplotlib
import matplotlib.pyplot as plt

### Repository

In [2]:
def CSVtoNumpyArray(rawdata):
    """
    Input: 
    rawdata = a csv file (insert name as a string)

    Output:
    two numpy matrices in a tuple
    
    Optimised 04/06/2020
    """
    data = pd.read_csv(rawdata)  #Reads the data in as a pandas object
    c = data.columns
    column = int(c[0])
    final_data1 = data.iloc[:column,:].values  #Sets data into a series of numpy arrays of strings
    final_data2 = data.iloc[column:,:].values  #1 is for the first matrix(loc) and 2 is for the second(flow)
    

    #Forms the matrix as a numpy array (easier to work with) instead of an list of lists of strings
    def string_to_integers(final_data):
        matrix = np.zeros(column)
        for j in range(column):
            string = final_data[j][0]
            string2 = string.split(" ")
            emptyarray = np.array([])
            for i in string2:
                if i != '':
                    emptyarray = np.append(emptyarray,i).astype(int)
            matrix = np.vstack((matrix,emptyarray))
        return matrix[1:]
    return string_to_integers(final_data1),string_to_integers(final_data2)

In [3]:
#small sized matrices(under 10x10) (quick on all methods)
matrix_size_4 = './data/made4.csv'
matrix_size_5 = './data/made5.csv'
matrix_size_6 = './data/made6.csv'
matrix_size_7 = './data/made7.csv'
matrix_size_8 = './data/made8.csv'
matrix_size_9 = './data/made9.csv'

matrixMade = ['./data/made4.csv', 
              './data/made5.csv', 
              './data/made6.csv', 
              './data/made7.csv', 
              './data/made8.csv', 
              './data/made9.csv']


#medium sized matrices(ranging from 10x10 to 30x30) (slow on deterministic methods, fast on heuristics)
matrix_size_10 = './data/tai10a.csv'
matrix_size_11 = './data/made11.csv'
matrix_size_12 = './data/tai12a.csv'
matrix_size_15 = './data/chr15a.csv' 
matrix_size_20 = './data/chr20a.csv'
matrix_size_26 = './data/bur26a.csv'

#large sized matrices(30x30 and bigger)(reasonably slow on the heuristics to a certain degree of accuracy)
matrix_size_40 = './data/tai40a.csv'
matrix_size_60 = './data/tai60.csv'
matrix_size_80 = './data/tai80.csv'
matrix_size_256 = './data/tai256c.csv'

datamatrix = CSVtoNumpyArray(matrix_size_4) # Decide the size of problem to run in the code (clue: 
                                                #the number in the original name is the size)
MatrixLoc = datamatrix[0]
MatrixFlow = datamatrix[1]

# Simulated Annealing Algorithm

### https://www.sciencedirect.com/science/article/pii/S0360835296002653 
### **Simulated Annealing for the Quadratic Assignment Problem: A Further Study by Tian Peng,  Wang Huanchen , Zhang Dongme**

This paper looks into the use of the Simulated Annealing Algorithm specifically for the Quadratic Assignment Problem.

In [4]:
def saQAP( T, e, L, alpha, name):
    """
    Input: 
    T - scalar value(large)
    e - scalar small value as a lower bound
    L - scalar large for number of iterations
    alpha - scalar between 0 and 1
    name - string to csv file
    
    Output:
    x - vector(1 by n) - final permutation
    fitnessQAP(x, D, F) - scalar total
    
    """
    
    MatrixLoc = CSVtoNumpyArray(name)[0]
    MatrixFlow = CSVtoNumpyArray(name)[1]
    n     = len(MatrixLoc)
    x= np.array(list(range(n)))
    
    while T > e:
        for i in range(L):
            num1, num2 = np.random.choice(n, 2, replace = False)
            fx      = fitnessQAP(x, MatrixLoc, MatrixFlow)
            y       = x.copy()
            temp    = y[num1]
            y[num1] = y[num2]
            y[num2] = temp
            fy      = fitnessQAP(y, MatrixLoc, MatrixFlow)
            if fy < fx:
                x  = y.copy()
                fx = fy.copy()
            elif np.random.rand() < np.exp(-(fy - fx)/T):
                x  = y.copy()
                fx = fy.copy()
        T = alpha*T
    return x, fitnessQAP(x, MatrixLoc, MatrixFlow)

In [5]:
def fitnessQAP(x, MatrixLoc, MatrixFlow):
    n     = len(x)
    total = 0
    for i in range(n):
        for k in range(n):
            if i != k:
                total += MatrixLoc[i][k]*MatrixFlow[x[i]][x[k]]
    return total

In [7]:
T     = 100000
e     = 1e-3
L     = 20
alpha = 0.99
saQAP( T, e, L, alpha, matrix_size_4)

(array([2, 3, 0, 1]), 790.0)

In [ ]:
def thirtytrials(optimal_parameters,T,name,known):
    final = np.array([])
    count95 = 0
    count99 = 0
    for i in range(30):
        ans = saQAP( T, optimal_parameters[0], optimal_parameters[1], optimal_parameters[2], name)[1]
        final = np.append(final,ans)
        #SR95
        if ans <= 1.05*known:
            count95+=1
        #sr99
        if ans <= 1.01*known:
            count99+=1
    return 100*count95/30, 100*count99/30